In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [13]:
import dgl
import torch
import gurobipy
import numpy as np

In [4]:
from src.data import get_model, load_instance

instance = load_instance('../data/raw/97_9.jl')
J = instance['jobs'][0]

model = get_model(list(range(J)), instance)

Restricted license - for non-production use only - expires 2024-10-28


In [14]:
model.NumVars

1746

In [15]:
model.NumIntVars

1746

In [19]:
from src.data import get_model, load_instance

instance = load_instance('../data/raw/97_9.jl')
J = instance['jobs'][0]

model = get_model(list(range(J)), instance, coupling=True)

In [20]:
model.NumVars, model.NumIntVars

(1842, 1746)

In [177]:
rs = torch.from_numpy(np.array(recurso_p[:T])).unsqueeze(0).repeat(1000, 1)
rs = torch.normal(rs, std=1)

for r in rs.numpy():
    m = get_model(list(range(J)), instance, coupling=True, recurso=r)

    A = m.getA().toarray()
    b = np.array(m.getAttr('rhs'))
    c = np.array(m.getAttr('obj'))

    # get only real (non-null) edges
    A_ = A.flatten()
    edges = np.indices(A.shape)  # cons -> vars
    edges = edges.reshape(edges.shape[0],-1)
    edges = edges[:,A_ != 0]
    # edges = torch.from_numpy(edges)

    edge_weights = A_[A_ != 0]

    soc_vars_mask = np.array(['soc' in v.getAttr(gurobipy.GRB.Attr.VarName) for v in m.getVars()])
    soc_vars = np.arange(soc_vars_mask.shape[0])[soc_vars_mask]
    var_vars = np.arange(soc_vars_mask.shape[0])[~soc_vars_mask]
    soc_edges_mask = np.isin(edges.T[:,1], soc_vars)

    var_edges = edges[:,~soc_edges_mask]
    soc_edges = edges[:,soc_edges_mask]
    
    # translate soc nodes index to 0-based
    soc_edges[1] = np.array(list(map(
        dict(zip(soc_vars, np.arange(soc_vars.shape[0]))).get,
        soc_edges[1]
    )))
    var_edges[1] = np.array(list(map(
        dict(zip(var_vars, np.arange(var_vars.shape[0]))).get,
        var_edges[1]
    )))

    g = dgl.heterograph({
        ('var', 'v2c', 'con'): (var_edges[1], var_edges[0]),
        ('con', 'c2v', 'var'): (var_edges[0], var_edges[1]),
        ('soc', 's2c', 'con'): (soc_edges[1], soc_edges[0]),
        ('con', 'c2s', 'soc'): (soc_edges[0], soc_edges[1]),
    })

    soc_edge_weights = edge_weights[soc_edges_mask]
    g.edges['s2c'].data['A'] = torch.from_numpy(soc_edge_weights)
    g.edges['c2s'].data['A'] = torch.from_numpy(soc_edge_weights)

    var_edge_weights = edge_weights[~soc_edges_mask]
    g.edges['v2c'].data['A'] = torch.from_numpy(var_edge_weights)
    g.edges['c2v'].data['A'] = torch.from_numpy(var_edge_weights)

    g.nodes['con'].data['b'] = torch.from_numpy(b)

    g.nodes['var'].data['c'] = torch.from_numpy(c[~soc_vars_mask])
    g.nodes['soc'].data['c'] = torch.from_numpy(c[soc_vars_mask])

In [171]:
b.shape

(5260,)

In [170]:
c.shape

(1842,)

In [176]:
c[~soc_vars_mask]

array([5., 5., 5., ..., 0., 0., 0.])

In [178]:
g

Graph(num_nodes={'con': 5260, 'soc': 96, 'var': 1746},
      num_edges={('con', 'c2s', 'soc'): 191, ('con', 'c2v', 'var'): 61755, ('soc', 's2c', 'con'): 191, ('var', 'v2c', 'con'): 61755},
      metagraph=[('con', 'soc', 'c2s'), ('con', 'var', 'c2v'), ('soc', 'con', 's2c'), ('var', 'con', 'v2c')])

In [160]:
m.NumBinVars, m.NumVars - m.NumBinVars

(1746, 96)

In [18]:
from src.dataset import ResourceDataset
from src.net import InstanceGCN

ds = ResourceDataset(instance, n_samples=3)

gs, rs = ds[0:2]
g = dgl.batch(gs)
g

Graph(num_nodes={'con': 10520, 'soc': 192, 'var': 3492},
      num_edges={('con', 'c2s', 'soc'): 382, ('con', 'c2v', 'var'): 123510, ('soc', 's2c', 'con'): 382, ('var', 'v2c', 'con'): 123510},
      metagraph=[('con', 'soc', 'c2s'), ('con', 'var', 'c2v'), ('soc', 'con', 's2c'), ('var', 'con', 'v2c')])

In [27]:
net = InstanceGCN(1, 1)
net(g)

tensor([[-0.1469],
        [-0.1469]], dtype=torch.float64, grad_fn=<DivBackward0>)